In [ ]:
from dbnomics_client import fetch_dataframe, fetch_dataframe_from_url

In [ ]:
fetch_dataframe(
    provider_code='Eurostat',
    dataset_code='nrg_134m',
    # limit=30,
)

In [ ]:
fetch_dataframe_from_url("http://localhost:5000/dataframe/Eurostat/met_road_gonu?dimension=nst07%3AGT15")